In [1]:
import nbimporter
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from hyperopt import hp, fmin, STATUS_OK, tpe, Trials
from hyperopt.pyll.base import scope

from Preprop import *
from Feature_En import *

In [2]:
d_path = 'datas'
df = data_preprocess(d_path, use_test=False)


D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x4 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [3]:
df = generate_features(df)

Feature_En.ipynb:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "            dataframes_profile[item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')\n",
Feature_En.ipynb:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        for featur in category_vars:\n",
Feature_En.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [4]:
df = df.loc[:, df.columns[3:]]
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], 
                                                    test_size=0.2, random_state=0, shuffle=False)

In [14]:
def objective_function(params, use_proba=True):
    model = CatBoostClassifier(n_estimators=50, random_state=0, **params)
    model.fit(X_train, y_train)
    if use_proba == True:
        y_pred = model.predict_proba(X_test)[:, 1]
        y_pred = y_pred.reshape(1, -1)
    elif use_proba == False:
        y_pred =  model.predict(X_test)
        y_pred = y_pred.reshape(1, -1)
    print(y_pred.shape)
    print(f"y_pred: {y_pred}")
    print("------------------------------")
    print(y_test.shape)
    print(f"y_test: {y_test}")
    metric = metrics.ndcg_score(y_test.values.reshape(1, -1), y_pred, k=20)
    return {'loss': -metric, 'status': STATUS_OK}

In [6]:
space = { 'depth': scope.int(hp.quniform('depth', 4, 10, 1)),
          'max_bin': scope.int(hp.quniform('max_bin', 5, 50, 1)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
          'random_strength': hp.loguniform('random_strength', np.log(1e-9), np.log(10)),
          'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
          'loss_function': hp.choice('loss_function', ['Logloss', 'CrossEntropy'])}

In [15]:
trials = Trials()
best_param = fmin(objective_function, space, algo=tpe.suggest, max_evals=10, 
                  trials=trials, rstate=np.random.default_rng(42))

0:	learn: 0.6568773	total: 863ms	remaining: 42.3s                                                                                                                                      

1:	learn: 0.6389309	total: 1.66s	remaining: 39.8s                                                                                                                                      

2:	learn: 0.6300163	total: 2.48s	remaining: 38.9s                                                                                                                                      

3:	learn: 0.6240329	total: 3.25s	remaining: 37.4s                                                                                                                                      

4:	learn: 0.6199224	total: 4.05s	remaining: 36.5s                                                                                                                                      

5:	learn: 0.6170527	total: 4.74s	remaining: 34.7s                          

In [16]:
print(best_param)

{'depth': 5.0, 'l2_leaf_reg': 7.910707579888384, 'learning_rate': 0.329353775876958, 'loss_function': 0, 'max_bin': 33.0, 'random_strength': 0.13694748986533703}


Лучшие параметры оказались следующие:
'depth': 5.0, 
'l2_leaf_reg': 7.910707579888384, 
'learning_rate': 0.329353775876958, 
'loss_function': 0, 
'max_bin': 33.0, 
'random_strength': 0.13694748986533703

Перенесём их в Fit.ipynb
